In [1]:
import pandas as pd

In [7]:
pd.set_option('display.max_colwidth', 3000)

In [2]:
df = pd.read_csv("datos-eph/usu_individual_T323.txt", sep=";")

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_4328\593093864.py:1: DtypeWarning: Columns (102) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("datos-eph/usu_individual_T323.txt", sep=";")


In [9]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df.dtypes)


CODUSU         object
ANO4            int64
TRIMESTRE       int64
NRO_HOGAR       int64
COMPONENTE      int64
H15             int64
REGION          int64
MAS_500        object
AGLOMERADO      int64
PONDERA         int64
CH03            int64
CH04            int64
CH05           object
CH06            int64
CH07            int64
CH08            int64
CH09            int64
CH10            int64
CH11            int64
CH12            int64
CH13            int64
CH14          float64
CH15            int64
CH15_COD      float64
CH16            int64
CH16_COD      float64
NIVEL_ED        int64
ESTADO          int64
CAT_OCUP        int64
CAT_INAC        int64
IMPUTA        float64
PP02C1          int64
PP02C2          int64
PP02C3          int64
PP02C4          int64
PP02C5          int64
PP02C6          int64
PP02C7          int64
PP02C8          int64
PP02E           int64
PP02H           int64
PP02I           int64
PP03C         float64
PP03D         float64
PP3E_TOT      float64
PP3F_TOT  